In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout, Conv2D, MaxPooling2D, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from sklearn.model_selection import KFold, train_test_split
from utils import utils

seed = 42
np.random.seed(seed)

In [ ]:
X_train, y_train = utils.get_train('data/IMG/', 'data/driving_log.csv')

In [ ]:
X_train.shape, y_train.shape

In [ ]:
img_height, img_width = X_train.shape[1], X_train.shape[2]
img_shape = X_train.shape[1:]

In [ ]:
def create_model(p=0.6, input_shape=(160, 320, 3)):
    model = Sequential()
    model.add(Lambda(lambda x: x / 255. - .5, input_shape=input_shape))
    # Convolutional 
    model.add(Conv2D(32, (3, 3), padding='same', activation ='relu'))
    model.add(MaxPooling2D(pool_size =(2, 2)))
    model.add(Conv2D(64, (3, 3), padding='same', activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Flatten
    model.add(Flatten())
    # Fully-Connection
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(p/2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(p))
    model.add(Dense(1))
    
    optimizer = Adam(1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])   
    return model 

In [ ]:
def train(X, y, epochs, batch_size):
    model = create_model(0.6, img_shape)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, shuffle=True)
    model.save('models/model.h5')
    return history

In [ ]:
def trainCV(X, y, nfolds=5, epochs=10, batch_size=32, model_dir="./"):
    # K-Fold cross-validation 
    kfold = KFold(n_splits=nfolds, shuffle=True, random_state=seed)
    # Pre-train
    pre_model = create_model(p=0.6, input_shape=img_shape) 
    pre_model.fit(X, y, batch_size=32, epochs=1, shuffle=True, verbose=1)
    pre_model.save_weights("pre-trained-model_w.h5")
    # Create directory for models
    utils.create_dir(model_dir)
    for k, (xi, cvi) in enumerate(kfold.split(X)):
        print("Fold {} of {}:".format(k+1, nfolds)) 
        model = create_model(p=0.6, input_shape=img_shape)
        pre_model.load_weights("pre-trained-model_w.h5")
        trX, cvX = X[xi], X[cvi] 
        trY, cvY = y[xi], y[cvi]
        # Callbacks
        modelname = "model" + "-fold-" + str(k+1) + "-{epoch:02d}-{val_acc:.3f}.h5"
        modelckpt = ModelCheckpoint(model_dir+"/" + modelname, monitor='val_acc', save_best_only=True)
        earlystop = EarlyStopping(monitor='val_acc', patience=2, verbose=0)
        callbacks = [earlystop, modelckpt]
        # Training
        model.fit(trX, trY, 
                  batch_size=32, 
                  epochs=epochs, 
                  shuffle=True, 
                  verbose=1,
                  validation_data=(cvX, cvY), 
                  callbacks=callbacks)        

In [ ]:
trainCV(X_train, y_train, nfolds=5, epochs=5, batch_size=32, model_dir="models/test")